In [1]:
from collections import defaultdict, OrderedDict
from datetime import datetime

import numpy as np
import pymongo
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN, KMeans, AgglomerativeClustering

In [2]:


class Settings:
    MONGODB_SERVER = "localhost"
    MONGODB_PORT = 27017
    MONGODB_DB = "beautiful_creature"
    MONGODB_INVESTOR_COLLECTION = "investors"
    MONGODB_PORTFOLIO_COLLECTION = "portfolios"

    def __getitem__(self, key):
        return getattr(self, key)


timestamp = datetime.now().strftime("%d-%m-%y")
settings = Settings()
connection = pymongo.MongoClient(
    settings['MONGODB_SERVER'],
    settings['MONGODB_PORT']
)
investor_collection_name = f"{settings['MONGODB_INVESTOR_COLLECTION']}_{timestamp}"
portfolio_collection_name = f"{settings['MONGODB_PORTFOLIO_COLLECTION']}_{timestamp}"
db = connection[settings['MONGODB_DB']]
investor_collection = db[investor_collection_name]
portfolio_collection = db[portfolio_collection_name]
investors = list(investor_collection.find({}))
investors = {i["UserName"]: i for i in investors}

In [3]:
portfols = list(portfolio_collection.find({}))

In [4]:
items = defaultdict(list)

In [5]:
for p in portfols:
    for i in p["items"]:
        if i["type"] == "Buying":
            try:
                items[i["company_ticker"]].append(round(float(i["invested"].replace("%", "")), 3))
            except ValueError as e:
                if "<0.01" in str(e):
                    pass
                else:
                    raise (e)

In [6]:

ticker_to_name_mapping = defaultdict(str)
for p in portfols:
    for i in p["items"]:
        ticker_to_name_mapping[i["company_ticker"]] = i.get("company_name", None)

In [7]:
# basic portfolio settings
TOP_N_BY_COUNTS = 100
TOP_N_BY_RATIO = 15


In [8]:
stats = []
for item in items:
    item_stat = {"item_name": item, "count": len(items[item]),
                 "mean": np.mean(items[item]), "std": np.std(items[item])}
    stats.append(item_stat)

df = pd.DataFrame.from_records(stats)
df["std/mean"] = df["std"] / df["mean"]
df = df.set_index("item_name")
df = df.sort_values("count", ascending=False).head(TOP_N_BY_COUNTS)

In [9]:
portfol_basic = df.sort_values("count", ascending=False).sort_values("std/mean").head(TOP_N_BY_RATIO)
portfol_basic["portfol_val"] = portfol_basic["mean"] * 100 / portfol_basic["mean"].sum()
portfol_basic.sort_values("portfol_val", ascending=False)
portfol_basic["company_name"] = portfol_basic.index.map(ticker_to_name_mapping)
portfol_basic.reset_index().sort_values("portfol_val", ascending=False).drop(["count", "mean", "std", "std/mean"],axis=1)

,item_name,portfol_val,company_name
13,AMD,10.629329,Advanced Micro Devices Inc
14,1810.HK,9.777061,Xiaomi Corp
8,PDD,9.409400,Pinduoduo Inc
4,CSIQ,9.202523,Canadian Solar Inc.
11,NTES,7.043327,NetEase Inc.
5,FXI,6.881766,None
3,SEDG,6.535106,SolarEdge Technologies
1,ENPH,6.277537,Enphase Energy Inc.
2,BLDP,5.655369,Ballard Power Systems Inc.
9,ARKG,5.585649,None


In [10]:
# weighted portfolio settings
TOP_N_BY_VALUE = 15
N_CLUSTERS = 7
TOP_INVESTORS_WITHIN_CLUSTER = 3

In [16]:
a_portfols = []
for p in portfols:
    for i in p["items"]:
        if i["type"] == "Buying":
            try:
                a_portfols.append((p["investor_name"], float(i["value"].replace("%", "")), i["company_ticker"]))
            except ValueError as e:
                if "<0.01" in str(e):
                    pass
                else:
                    raise (e)

In [12]:
a_df = pd.DataFrame(data=a_portfols, columns=["investor", "percent", "ticker"])
a_df = a_df.pivot_table(index="investor", columns="ticker").fillna(0)
a_df["cluster"] = KMeans(n_clusters=N_CLUSTERS).fit_predict(a_df)
fields = ["Copiers", "WeeksSinceRegistration", "DailyDD", "WeeklyDD", "RiskScore", "Gain"]

for investor_name in a_df.index:
    for field in fields:
        a_df.loc[investor_name, field] = investors[investor_name][field]
a_df_short = a_df.drop("percent", axis=1)
a_df_short

,cluster,Copiers,WeeksSinceRegistration,DailyDD,WeeklyDD,RiskScore,Gain
ticker,,,,,,,
investor,,,,,,,
2BSmart,3,323.0,226.0,-4.38,-7.63,4.0,37.65
ABDUCT,4,584.0,95.0,-3.12,-3.12,3.0,26.57
ALnayef,4,1708.0,263.0,-2.21,-3.19,3.0,15.60
Abbroush,4,514.0,63.0,-1.32,-1.93,3.0,16.17
Aguero1010,4,1101.0,63.0,-4.26,-5.58,4.0,80.07
...,...,...,...,...,...,...,...
traderengeng,3,116.0,67.0,-3.63,-5.55,3.0,47.27
vidinho,0,133.0,627.0,-3.51,-4.45,4.0,27.47


In [13]:
top_investors = a_df_short.groupby("cluster")["Gain"].nlargest(TOP_INVESTORS_WITHIN_CLUSTER).reset_index()["investor"]
a_df_short.loc[top_investors]

,cluster,Copiers,WeeksSinceRegistration,DailyDD,WeeklyDD,RiskScore,Gain
ticker,,,,,,,
investor,,,,,,,
vidinho,0,133.0,627.0,-3.51,-4.45,4.0,27.47
balticseal,0,1341.0,99.0,-2.40,-2.99,3.0,17.15
chiay0327,0,762.0,236.0,-1.90,-2.52,3.0,10.05
myhungetoro,1,388.0,98.0,-2.09,-3.29,3.0,5.48
EliteVol,2,176.0,97.0,-4.15,-10.59,5.0,22.25
CPHequities,3,19258.0,223.0,-4.52,-11.12,4.0,92.29
Financial_man,3,883.0,201.0,-4.20,-9.10,4.0,84.20
MarianoPardo,3,9151.0,403.0,-4.34,-8.02,5.0,73.12


In [14]:
portfol_weight = a_df.loc[top_investors]["percent"].sum(axis=0)
portfol_weight = portfol_weight[portfol_weight != 0]
portfol_weight = portfol_weight * 100 / portfol_weight.sum()
portfol_weight = portfol_weight.sort_values(ascending=False)[:TOP_N_BY_VALUE]
portfol_weight = portfol_weight * 100 / portfol_weight.sum()
portfol_weight

ticker
TLT     21.986127
SVXY    10.484592
VTI      8.483194
SHV      7.904869
IVV      5.922966
MSFT     5.614309
IAU      5.159446
BABA     5.156197
FB       5.120457
AMZN     4.391052
GLD      4.353688
QQQ      4.262716
IEF      4.075897
AMD      3.642153
V        3.442338
dtype: float64

In [15]:

portfol_weight_new = portfol_weight.to_frame("portfol_val")
portfol_weight_new["company_name"] = portfol_weight_new.index.map(ticker_to_name_mapping)
portfol_weight_new = portfol_weight_new.reset_index()
portfol_weight_new

,ticker,portfol_val,company_name
0,TLT,21.986127,None
1,SVXY,10.484592,None
2,VTI,8.483194,None
3,SHV,7.904869,None
4,IVV,5.922966,None
5,MSFT,5.614309,Microsoft
6,IAU,5.159446,None
7,BABA,5.156197,Alibaba
8,FB,5.120457,Facebook
9,AMZN,4.391052,Amazon
